Question 11
(Optional programming exercises)

    Implement a histogram equalization function. If using Matlab, compare your implementation with Matlab’s built-in function.
    Implement a median filter. Add different levels and types of noise to an image and experiment with different sizes of support for the median filter. As before, compare your implementation with Matlab’s.
    Implement the non-local means algorithm. Try different window sizes. Add different levels of noise and see the influence of it in the need for larger or smaller neighborhoods. (Such block operations are easy when using Matlab, see for example the function at http://www.mathworks.com/help/images/ref/blockproc.html). Compare your results with those available in IPOL as demonstrated in the video lectures. http://www.ipol.im/pub/art/2011/bcm_nlm/
    Consider an image and add to it random noise. Repeat this N times, for different values of N, and add the resulting images. What do you observe?
    Implement the basic color edge detector. What happens when the 3 channels are equal?
    Take a video and do frame-by-frame histogram equalization and run the resulting video. Now consider a group of frames as a large image and do histogram equalization for all of them at once. What looks better? See this example on how to read and handle videos in Matlab:

    xyloObj = VideoReader('xylophone.mp4');

    nFrames = xyloObj.NumberOfFrames;
    vidHeight = xyloObj.Height;
    vidWidth = xyloObj.Width;

    % Preallocate movie structure.
    mov(1:nFrames) = struct('cdata', zeros(vidHeight, vidWidth, 3, 'uint8'), 'colormap', []);

    % Read one frame at a time.
    for k = 1 : nFrames
        im = read(xyloObj, k);

        % here we process the image im

        mov(k).cdata = im;
    end

    % Size a figure based on the video's width and height.
    hf = figure;
    set(hf, 'position', [150 150 vidWidth vidHeight])

    % Play back the movie once at the video's frame rate.
    movie(hf, mov, 1, xyloObj.FrameRate);


    Take a video and do frame-by-frame non-local means denoising. Repeat but now using a group of frames as a large image. This allows you for example to find more matching blocks (since you are searching across frames). Compare the results. What happens if now you use 3D spatio-temporal blocks, e.g., 5×5×3 blocks and consider the group of frames as a 3D image? Try this and compare with previous results.
    Search for “camouflage artist liu bolin.” Do you think you can use the tools you are learning to detect him?


## Part 1

Implement a histogram equalization function. If using Matlab, compare your implementation with Matlab’s built-in function.
    

In [ ]:
from __future__ import division, print_function

import skimage
import numpy as np
from matplotlib import pyplot as plt
from skimage import img_as_float, img_as_ubyte
from skimage import io

%matplotlib inline

In [ ]:
img = io.imread("almonds.jpeg")
img = img[:,:,1]
# img = img_as_float(img)
# plt.imshow(img, cmap="gray")

img2 = img_as_ubyte(img_as_float(img)**3.0) # darken the image

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,10))
axes[0].imshow(img, cmap="gray")
axes[1].imshow(img2, cmap="gray")
plt.tight_layout()


In [ ]:
# histogram equalization
vals, counts = np.unique(img2, return_counts=True)
new_vals = (255*np.cumsum(counts)/np.sum(counts)).astype(int)
lut = dict(zip(vals, new_vals))

img3 = np.zeros_like(img2)
rows = img2.shape[0]
cols = img2.shape[1]

for r in range(rows):
    for c in range(cols):
        img3[r,c] = lut[img2[r,c]]
   


In [ ]:
from skimage import exposure
img_eq = img_as_ubyte(exposure.equalize_hist(img2))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15,10))
axes[0,0].imshow(img, cmap="gray")
axes[0,0].set_title("Original")
axes[1,0].hist(img.ravel(), bins=255);
axes[0,1].imshow(img2, cmap="gray")
axes[0,1].set_title("Darkened")
axes[1,1].hist(img2.ravel(), bins=255);
axes[0,2].imshow(img3, cmap="gray")
axes[0,2].set_title("Manual Histogram Equalization")
axes[1,2].hist(img3.ravel(), bins=255);
axes[0,3].imshow(img_eq, cmap="gray")
axes[0,3].set_title("sklearn Histogram Equalization")
axes[1,3].hist(img_eq.ravel(), bins=255);
plt.tight_layout()

## Part 2

Implement a median filter. Add different levels and types of noise to an image and experiment with different sizes of support for the median filter. As before, compare your implementation with Matlab’s.
    

In [ ]:
def add_noise(img, delta_max, delta_prob):
    noise = np.random.randint(-delta_max,delta_max+1,size=img.shape)
    mask = np.random.rand(img.shape[0], img.shape[1])
    noise = noise * (mask < delta_prob) # keeps noise with prob < delta_prob
    im_noise = img + noise
    return np.clip(im_noise,0,255)

def add_salt_pepper(img, prob):
    noisy_image = img.copy()
    noise = np.random.random(img.shape)
    noisy_image[noise > (1-prob/2)] = 255
    noisy_image[noise < prob/2] = 0
    return noisy_image

In [ ]:
img4 = np.zeros_like(img)
rows = img4.shape[0]
cols = img4.shape[1]

# img5 = add_noise(img, 100, 0.95)
img5 = add_salt_pepper(img, 0.01)
win = 3 # odd number only
offset = np.floor(win/2).astype(int)
# print(offset)

for r in range(offset, rows-offset):
    for c in range(offset, cols-offset):
        neighbors = img5[(r-offset):(r+offset+1), (c-offset):(c+offset+1)].ravel()
        # print(neighbors)
        median_val = np.sort(neighbors)[np.ceil(win**2/2.0).astype(int)]
        img4[r,c] = median_val
        
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,10))
axes[0].imshow(img, cmap="gray")
axes[0].set_title("Original")
axes[1].imshow(img5, cmap="gray")
axes[1].set_title("Noisy Image");
axes[2].imshow(img4, cmap="gray")
axes[2].set_title("manual Median Filtered");

In [ ]:
from skimage.filters.rank import median
from skimage.morphology import disk

img6 = median(img4, disk(1))

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,10))
axes[0].imshow(img, cmap="gray")
axes[0].set_title("Original")
axes[1].imshow(img5, cmap="gray")
axes[1].set_title("Noisy Image");
axes[2].imshow(img6, cmap="gray")
axes[2].set_title("sklearn Median Filtered");


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,10))
ax.imshow(img5-img4, cmap="gray")
ax.set_title("sklearn - manual");

## Part 3

Implement the non-local means algorithm. Try different window sizes. Add different levels of noise and see the influence of it in the need for larger or smaller neighborhoods. (Such block operations are easy when using Matlab, see for example the function at http://www.mathworks.com/help/images/ref/blockproc.html). Compare your results with those available in IPOL as demonstrated in the video lectures. http://www.ipol.im/pub/art/2011/bcm_nlm/
    

In [ ]:
def nonlocal_avg(img, block, win, rmse_threshold):
    pix_new = 0
    offset = np.floor(win/2).astype(int)
    count = 0
    sum = 0
    rmse_list = []
    
    for r in range(offset, rows-offset):
        for c in range(offset, cols-offset):
            blk = img[(r-offset):(r+offset+1),(c-offset):(c+offset+1)]
            # print(r,c,block.ravel(),blk.ravel())
            sse = np.sum((block.ravel() - blk.ravel())**2)
            rmse = np.sqrt(sse/win**2)
            rmse_list.append(rmse)
            if rmse < rmse_threshold:
                sum = sum + img[r,c]
                count = count + 1
    
    pix_new = int(sum/float(count))
    # print("rmse mean, sd, min, max: ", np.mean(rmse_list), np.std(rmse_list), np.min(rmse_list), np.max(rmse_list))
    return pix_new

In [ ]:
def nonlocal_pix_avg(img_blocks, block_blocks, rmse_threshold):
    pix_new = 0
    center_pix = int(img_blocks.shape[2]/2) + 1
    
    se = (img_blocks - block_blocks)**2
    sse = np.sum(np.sum(se, axis=2), axis=1)
    mse = sse/(se.shape[1]*se.shape[2])
    rmse = np.sqrt(mse)
    
    # print("rmse mean, sd, min, max: ", np.mean(rmse), np.std(rmse), np.min(rmse), np.max(rmse))

    mask = rmse < rmse_threshold
    
    pix_new = np.mean(img_blocks[mask, center_pix, center_pix]).astype(int)

    return pix_new

def img_as_block_array(img, win):
    rows = img.shape[0]
    cols = img.shape[1]
    img_blocks = np.zeros(((rows-win+1)*(cols-win+1),win,win))
    offset = np.floor(win/2).astype(int)
    
    idx = 0
    for r in range(offset, rows-offset):
        for c in range(offset, cols-offset):
            img_blocks[idx,:,:] = img[(r-offset):(r+offset+1),(c-offset):(c+offset+1)]
            idx=idx+1
            
    return img_blocks

def block_as_block_array(block, img_blocks):
    block_blocks = np.zeros_like(img_blocks)
    rows_blocks = block_blocks.shape[1]
    cols_blocks = block_blocks.shape[2]
        
    for rb in range(rows_blocks):
        for cb in range(cols_blocks):
            block_blocks[:,rb,cb] = block[rb,cb]

    return block_blocks

def random_sample_img_blocks(img_blocks, n_blocks):
    n2 = img_blocks.shape[0]
    if n2 <= n_blocks:
        return img_blocks
    else:
        n = np.random.randint(0, n2, n_blocks) # sampling with replacement
        return img_blocks[n,:,:]

def nonlocal_avg(img, win, rmse_threshold, n_blocks):
    img_avg = np.zeros_like(img)
    rows = img_avg.shape[0]
    cols = img_avg.shape[1]
    offset = np.floor(win/2).astype(int)

    img_blocks = img_as_block_array(img, win)

    for r in range(offset, rows-offset):
        # sys.stdout.write(str(r)+", ") 

        for c in range(offset, cols-offset):
            block = img[(r-offset):(r+offset+1),(c-offset):(c+offset+1)]
        
            img_blocks_subset = random_sample_img_blocks(img_blocks, n_blocks)
            img_blocks_subset[0,:,:] = block # Make sure the target block is included.
        
            block_blocks = block_as_block_array(block, img_blocks_subset)

            img_avg[r,c] = nonlocal_pix_avg(img_blocks_subset, block_blocks, rmse_threshold)
    return img_avg

In [ ]:
import sys
import random
import time

img_original = io.imread("../lena512color.tiff")
img_original = img_original[:,:,1] # take the green channel as intensity
# img_original = img_original[200:300, 200:400]
img_noisy = add_salt_pepper(img_original, 0.01)
win = 9
%timeit img_blocks = img_as_block_array(img_noisy, win)
print("img_blocks.shape", img_blocks.shape)

n_blocks = 500
%timeit img_blocks_subset = random_sample_img_blocks(img_blocks, n_blocks)
print("img_blocks_subset.shape", img_blocks_subset.shape)

block = img_noisy[0:win,0:win]
%timeit block_blocks = block_as_block_array(block, img_blocks_subset)
print("block_blocks.shape", block_blocks.shape)

rmse_threshold = 30
%timeit pix = nonlocal_pix_avg(img_blocks_subset, block_blocks, rmse_threshold)
print("pix", pix)



In [ ]:
%timeit idxes = np.arange(img_blocks.shape[0]).astype(int)
print("len(idxes)", len(idxes))

n = n_blocks
%timeit n = np.min([n_blocks, len(idxes)])
print("n", n)


# print("len(idxes_subset)", len(idxes_subset))

In [ ]:
%timeit idxes_subset = random.sample(idxes, n)

In [ ]:
%timeit i = np.random.randint(0, img_blocks.shape[0], n)

In [ ]:
%timeit img_blocks_subset = img_blocks[idxes_subset,:,:]

In [ ]:
i = np.random.randint(0, img_blocks.shape[0], n)
%timeit img_blocks_subset = img_blocks[i,:,:]

In [ ]:
range(30,70,2)

In [ ]:
import sys
import random
import time

img_original = io.imread("../lena512color.tiff")
img_original = img_original[:,:,1] # take the green channel as intensity
img_original = img_original[10:500, 10:500]

img_noisy = add_salt_pepper(img_original, 0.01)
# img2 = add_noise(img, delta_max=50, delta_prob=0.20)

win = 5 # odd number only
rmse_threshold = 30
n_blocks = 50

imgs_reconstructed = []
params = []

for win in [5]:
    for rmse_threshold in range(30,71,2):
        for n_blocks in [400, 800, 1600]:
            t = time.strftime("%Y%m%d_%H%M")
            print(win, rmse_threshold, n_blocks, t)
            params.append([win, rmse_threshold, n_blocks])
            imgs_reconstructed.append(nonlocal_avg(img_noisy, win, rmse_threshold, n_blocks))

n_col=3
n_row = np.ceil(len(params)/float(n_col)).astype(int)
fig, axes = plt.subplots(nrows=n_row, ncols=n_col, figsize=(15,n_row*5))

i = 0
for r in range(n_row):
    for c in range(n_col):
        if i < len(params):
            axes[r,c].imshow(imgs_reconstructed[i], cmap="gray")
            axes[r,c].set_title("win, thresh, n:" + str(params[i]))
        i = i + 1
plt.tight_layout()
filename = "nonlocal_average_" + time.strftime("%Y%m%d_%H%M") + ".png"
fig.savefig(filename, bbox_inches='tight')

In [ ]:
import sys
import random
import time

img_original = io.imread("../lena512color.tiff")
img_original = img_original[:,:,1] # take the green channel as intensity
img_original = img_original[10:500, 10:500]

img_noisy = add_salt_pepper(img_original, 0.01)
# img2 = add_noise(img, delta_max=50, delta_prob=0.20)

win = 5 # odd number only
rmse_threshold = 30
n_blocks = 50

img_reconstructed = nonlocal_avg(img_noisy, win=5, rmse_threshold=50, n_blocks=1600)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,15))
axes[0].imshow(img_original, cmap="gray")
axes[0].set_title("Original")
axes[1].imshow(img_noisy, cmap="gray")
axes[1].set_title("Noisy")
axes[2].imshow(img_reconstructed, cmap="gray")
axes[2].set_title("Reconstructed, win=5, rmse=50, n=1600")

plt.tight_layout()

filename = "nonlocal_average_" + time.strftime("%Y%m%d_%H%M") + ".png"
fig.savefig(filename, bbox_inches='tight')

In [ ]:
rmses = []

i = 0
for im in imgs_reconstructed:
    rows = img_original.shape[0]
    cols = img_original.shape[1]
    offset = 10
    se = (img_original[offset:-offset,offset:-offset] - im[offset:-offset,offset:-offset])**2
    mse = np.mean(se)
    rmses.append(np.sqrt(mse))

            

In [ ]:
idx = np.where(rmses == np.min(rmses))[0][0]
print(idx, rmses[idx], params[idx])

## Part 4

Consider an image and add to it random noise. Repeat this N times, for different values of N, and add the resulting images. What do you observe?
    

In [ ]:
img = io.imread("../lena512color.tiff")
img = img[:,:,1] # take the green channel as intensity

N = 16384
# N = 512
prob = 0.75
# prob = 0.001
img2 = np.zeros(img.shape, dtype=np.uint64)

for im in range(N):
    img2 = img2 + add_salt_pepper(img, prob).astype(np.uint64)
    if im % 500 == 0:
        sys.stdout.write(str(im)+", ") 

mn = np.min(img2)
mx = np.max(img2)
img2 = img_as_ubyte((img2.astype(float) - mn)/float(mx-mn))
# img2 = (img2/float(N)).astype(np.uint8)
print(np.min(img), np.max(img))
print(np.min(img2), np.max(img2))
# print(np.min(img-img2), np.max(np.abs(img-img2)))

delta = img_as_ubyte(np.abs(img_as_float(img) - img_as_float(img2)))
print(np.min(delta), np.max(delta))

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15,10))
axes[0].imshow(img, cmap="gray")
axes[0].set_title("Original")
axes[1].imshow(add_salt_pepper(img, prob), cmap="gray")
axes[1].set_title("Typical Noisy Image")
axes[2].imshow(img2, cmap="gray")
axes[2].set_title("Reconstructed");

fig.savefig('add_noisy_images.png', bbox_inches='tight')

Implement the basic color edge detector. What happens when the 3 channels are equal?
    

Take a video and do frame-by-frame histogram equalization and run the resulting video. Now consider a group of frames as a large image and do histogram equalization for all of them at once. What looks better? See this example on how to read and handle videos in Matlab